In [1]:
!gsutil -m cp gs://project_shm/Image_224.npy ./

Copying gs://project_shm/Image_224.npy...
/ [1/1 files][  2.4 GiB/  2.4 GiB] 100% Done  84.3 MiB/s ETA 00:00:00           
Operation completed over 1 objects/2.4 GiB.                                      


In [2]:
!gsutil -m cp gs://project_shm/Label_224.npy ./

Copying gs://project_shm/Label_224.npy...
/ [1/1 files][131.3 KiB/131.3 KiB] 100% Done                                    
Operation completed over 1 objects/131.3 KiB.                                    


In [9]:
!sudo pip install numpy

In [8]:
!sudo pip install -U keras-tuner

     |████████████████████████████████| 92kB 4.8MB/s eta 0:00:011
     |████████████████████████████████| 51kB 10.4MB/s eta 0:00:01
     |████████████████████████████████| 86.3MB 136kB/s  eta 0:00:01
  Created wheel for tabulate: filename=tabulate-0.8.6-cp35-none-any.whl size=30505 sha256=bba3d3683f8ab069426bb6410e2fafdc7b5d920731d8ff046e92c33a2d7fc98b
  Stored in directory: /root/.cache/pip/wheels/9c/9b/f4/eb243fdb89676ec00588e8c54bb54360724c06e7fafe95278e
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp35-none-any.whl size=17907 sha256=77ffe4608fe82a0756a091b33c7167e8e882c26bb2f0584efbf2c609e3fbaffd
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built tabulate terminaltables
ERROR: tensorflow 2.0.0 has requirement numpy<2.0,>=1.16.0, but you'll have numpy 1.15.1 which is incompatible.


In [10]:
import tensorflow as tf
import tensorflow.keras.layers as Layers
import tensorflow.keras.activations as Actications
import tensorflow.keras.models as Models
import tensorflow.keras.optimizers as Optimizer
import tensorflow.keras.metrics as Metrics
import tensorflow.keras.utils as Utils
from tensorflow import keras
import matplotlib.pyplot as plot
import numpy as np
#from imblearn.over_sampling import BorderlineSMOTE
import random 
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

In [11]:
np_Images = np.load('./Image_224.npy') #converting the list of images to numpy array.
np_Labels = np.load('./Label_224.npy')

In [12]:
print("Shape of Images:",np_Images.shape)
print("Shape of Labels:",np_Labels.shape)

Shape of Images: (16785, 224, 224, 3)
Shape of Labels: (16785,)


In [13]:
np.bincount(np_Labels)

array([9955,  229,  766, 2061,  426, 3348])

In [14]:
np_Images=tf.keras.applications.resnet50.preprocess_input(np_Images)

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X_train_tmp, X_test, y_train_tmp, y_test = train_test_split(np_Images, np_Labels, test_size=0.1, random_state=1, stratify=np_Labels)

In [17]:
X_train, X_val, y_train, y_val = train_test_split(X_train_tmp, y_train_tmp, test_size=0.11, random_state=1 ,stratify=y_train_tmp)

In [18]:
np.bincount(y_train)

array([7973,  183,  613, 1651,  342, 2682])

In [19]:
len(X_train), len(y_train)

(13444, 13444)

In [20]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)
np.bincount(y_train),np.bincount(y_test)

(13444, 224, 224, 3) (1679, 224, 224, 3) (13444,) (1679,)


(array([7973,  183,  613, 1651,  342, 2682]),
 array([996,  23,  77, 206,  42, 335]))

In [21]:
from kerastuner import HyperModel
from kerastuner import HyperParameters
hp = HyperParameters()

class MyHyperModel(HyperModel):

    def __init__(self, num_classes):
        self.num_classes = num_classes

    def build(self, hp):
        base_model = tf.keras.applications.ResNet50(input_shape=(224,224,3),
                                            include_top=False,   
                                            weights='imagenet')
        model = tf.keras.Sequential([
            base_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(units=hp.Int('uints', min_value=32, max_value=512, step=32), activation=None),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Dense(self.num_classes, activation='softmax') 
            ])
        model.compile(
            optimizer=keras.optimizers.Adam(
                hp.Choice('learning_rate',
                          values=[1e-2, 1e-3, 1e-4])),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])
        return model


hypermodel = MyHyperModel(num_classes=6)

In [24]:
tuner = RandomSearch(
    hypermodel,
    objective='val_accuracy',
    max_trials=10,
    directory='my_dir1',
    project_name='helloworld')

tuner.search(X_train, y_train,
             epochs=10,
             validation_data=(X_val, y_val))

INFO:tensorflow:Reloading Oracle from my_dir1/helloworld/oracle.json
INFO:tensorflow:Reloading Tuner from my_dir1/helloworld/tuner0.json
Train on 13444 samples, validate on 1662 samples
Epoch 1/10
13444/13444 [==============================] - ETA: 1:13:52 - loss: 2.2158 - accuracy: 0.187 - ETA: 37:27 - loss: 2.0556 - accuracy: 0.2188  - ETA: 25:19 - loss: 1.9319 - accuracy: 0.218 - ETA: 19:15 - loss: 1.8675 - accuracy: 0.257 - ETA: 15:36 - loss: 1.7572 - accuracy: 0.318 - ETA: 13:10 - loss: 1.6973 - accuracy: 0.349 - ETA: 11:26 - loss: 1.5941 - accuracy: 0.392 - ETA: 10:08 - loss: 1.5398 - accuracy: 0.414 - ETA: 9:07 - loss: 1.4847 - accuracy: 0.451 - ETA: 8:18 - loss: 1.4142 - accuracy: 0.48 - ETA: 7:38 - loss: 1.3726 - accuracy: 0.50 - ETA: 7:05 - loss: 1.3252 - accuracy: 0.53 - ETA: 6:36 - loss: 1.2659 - accuracy: 0.56 - ETA: 6:12 - loss: 1.2358 - accuracy: 0.57 - ETA: 5:51 - loss: 1.1856 - accuracy: 0.59 - ETA: 5:33 - loss: 1.1417 - accuracy: 0.61 - ETA: 5:17 - loss: 1.1104 - accu

Train on 13444 samples, validate on 1662 samples
Epoch 1/10
13444/13444 [==============================] - ETA: 1:19:13 - loss: 2.1903 - accuracy: 0.093 - ETA: 40:07 - loss: 1.7870 - accuracy: 0.3125  - ETA: 27:06 - loss: 1.5653 - accuracy: 0.427 - ETA: 20:34 - loss: 1.3863 - accuracy: 0.523 - ETA: 16:40 - loss: 1.3141 - accuracy: 0.550 - ETA: 14:03 - loss: 1.2205 - accuracy: 0.583 - ETA: 12:11 - loss: 1.0936 - accuracy: 0.633 - ETA: 10:47 - loss: 1.0590 - accuracy: 0.648 - ETA: 9:42 - loss: 0.9992 - accuracy: 0.663 - ETA: 8:49 - loss: 0.9627 - accuracy: 0.68 - ETA: 8:07 - loss: 0.9632 - accuracy: 0.69 - ETA: 7:31 - loss: 0.9477 - accuracy: 0.69 - ETA: 7:01 - loss: 0.9102 - accuracy: 0.70 - ETA: 6:35 - loss: 0.8940 - accuracy: 0.71 - ETA: 6:12 - loss: 0.8757 - accuracy: 0.71 - ETA: 5:53 - loss: 0.8511 - accuracy: 0.72 - ETA: 5:35 - loss: 0.8534 - accuracy: 0.72 - ETA: 5:20 - loss: 0.8598 - accuracy: 0.72 - ETA: 5:06 - loss: 0.8506 - accuracy: 0.73 - ETA: 4:53 - loss: 0.8554 - accuracy:

Train on 13444 samples, validate on 1662 samples
Epoch 1/10
13444/13444 [==============================] - ETA: 1:14:40 - loss: 2.3542 - accuracy: 0.062 - ETA: 37:51 - loss: 1.7590 - accuracy: 0.3750  - ETA: 25:34 - loss: 1.6755 - accuracy: 0.458 - ETA: 19:26 - loss: 1.5116 - accuracy: 0.531 - ETA: 15:45 - loss: 1.3778 - accuracy: 0.575 - ETA: 13:18 - loss: 1.3865 - accuracy: 0.583 - ETA: 11:32 - loss: 1.3216 - accuracy: 0.611 - ETA: 10:13 - loss: 1.3002 - accuracy: 0.625 - ETA: 9:12 - loss: 1.2759 - accuracy: 0.638 - ETA: 8:23 - loss: 1.1929 - accuracy: 0.65 - ETA: 7:42 - loss: 1.1974 - accuracy: 0.66 - ETA: 7:09 - loss: 1.1669 - accuracy: 0.66 - ETA: 6:40 - loss: 1.1250 - accuracy: 0.67 - ETA: 6:16 - loss: 1.0959 - accuracy: 0.67 - ETA: 5:54 - loss: 1.0586 - accuracy: 0.68 - ETA: 5:36 - loss: 1.0507 - accuracy: 0.69 - ETA: 5:19 - loss: 1.0364 - accuracy: 0.70 - ETA: 5:05 - loss: 1.0198 - accuracy: 0.70 - ETA: 4:52 - loss: 0.9995 - accuracy: 0.70 - ETA: 4:40 - loss: 0.9915 - accuracy:

Train on 13444 samples, validate on 1662 samples
Epoch 1/10
13444/13444 [==============================] - ETA: 1:13:43 - loss: 1.8201 - accuracy: 0.312 - ETA: 37:22 - loss: 1.6263 - accuracy: 0.4531  - ETA: 25:15 - loss: 2.0656 - accuracy: 0.458 - ETA: 19:12 - loss: 1.8387 - accuracy: 0.492 - ETA: 15:34 - loss: 2.0350 - accuracy: 0.481 - ETA: 13:08 - loss: 2.0049 - accuracy: 0.489 - ETA: 11:24 - loss: 1.8679 - accuracy: 0.517 - ETA: 10:06 - loss: 2.0124 - accuracy: 0.527 - ETA: 9:06 - loss: 1.9559 - accuracy: 0.531 - ETA: 8:17 - loss: 1.9052 - accuracy: 0.54 - ETA: 7:37 - loss: 1.8581 - accuracy: 0.55 - ETA: 7:04 - loss: 1.7942 - accuracy: 0.55 - ETA: 6:36 - loss: 1.7327 - accuracy: 0.56 - ETA: 6:12 - loss: 1.7074 - accuracy: 0.57 - ETA: 5:51 - loss: 1.6768 - accuracy: 0.57 - ETA: 5:32 - loss: 1.7162 - accuracy: 0.58 - ETA: 5:16 - loss: 1.6888 - accuracy: 0.58 - ETA: 5:02 - loss: 1.7657 - accuracy: 0.58 - ETA: 4:49 - loss: 1.7325 - accuracy: 0.58 - ETA: 4:37 - loss: 1.7114 - accuracy:

Train on 13444 samples, validate on 1662 samples
Epoch 1/10
13444/13444 [==============================] - ETA: 1:13:27 - loss: 2.0696 - accuracy: 0.156 - ETA: 37:15 - loss: 2.4270 - accuracy: 0.2969  - ETA: 25:11 - loss: 2.3080 - accuracy: 0.333 - ETA: 19:08 - loss: 2.2011 - accuracy: 0.398 - ETA: 15:31 - loss: 2.1238 - accuracy: 0.381 - ETA: 13:06 - loss: 2.0859 - accuracy: 0.395 - ETA: 11:22 - loss: 1.9472 - accuracy: 0.428 - ETA: 10:04 - loss: 1.8645 - accuracy: 0.449 - ETA: 9:04 - loss: 1.8047 - accuracy: 0.461 - ETA: 8:15 - loss: 1.7584 - accuracy: 0.48 - ETA: 7:36 - loss: 1.7602 - accuracy: 0.48 - ETA: 7:02 - loss: 1.7300 - accuracy: 0.49 - ETA: 6:34 - loss: 1.6966 - accuracy: 0.50 - ETA: 6:10 - loss: 1.6553 - accuracy: 0.50 - ETA: 5:50 - loss: 1.6293 - accuracy: 0.50 - ETA: 5:31 - loss: 1.5932 - accuracy: 0.51 - ETA: 5:15 - loss: 1.5975 - accuracy: 0.51 - ETA: 5:01 - loss: 1.5970 - accuracy: 0.50 - ETA: 4:48 - loss: 1.5914 - accuracy: 0.50 - ETA: 4:36 - loss: 1.6086 - accuracy:

Train on 13444 samples, validate on 1662 samples
Epoch 1/10
13444/13444 [==============================] - ETA: 1:13:13 - loss: 2.6883 - accuracy: 0.031 - ETA: 37:08 - loss: 2.1158 - accuracy: 0.2656  - ETA: 25:06 - loss: 1.8998 - accuracy: 0.364 - ETA: 19:04 - loss: 1.6812 - accuracy: 0.437 - ETA: 15:28 - loss: 1.5362 - accuracy: 0.493 - ETA: 13:03 - loss: 1.4887 - accuracy: 0.526 - ETA: 11:20 - loss: 1.3619 - accuracy: 0.584 - ETA: 10:03 - loss: 1.3157 - accuracy: 0.605 - ETA: 9:02 - loss: 1.2903 - accuracy: 0.621 - ETA: 8:14 - loss: 1.2569 - accuracy: 0.63 - ETA: 7:34 - loss: 1.2130 - accuracy: 0.65 - ETA: 7:01 - loss: 1.1893 - accuracy: 0.66 - ETA: 6:34 - loss: 1.1622 - accuracy: 0.67 - ETA: 6:10 - loss: 1.1458 - accuracy: 0.68 - ETA: 5:49 - loss: 1.1170 - accuracy: 0.68 - ETA: 5:30 - loss: 1.0960 - accuracy: 0.69 - ETA: 5:14 - loss: 1.0732 - accuracy: 0.70 - ETA: 5:00 - loss: 1.0596 - accuracy: 0.70 - ETA: 4:47 - loss: 1.0463 - accuracy: 0.71 - ETA: 4:36 - loss: 1.0454 - accuracy:

Train on 13444 samples, validate on 1662 samples
Epoch 1/10
13444/13444 [==============================] - ETA: 1:13:22 - loss: 3.0314 - accuracy: 0.031 - ETA: 37:12 - loss: 2.7827 - accuracy: 0.2188  - ETA: 25:09 - loss: 2.5366 - accuracy: 0.322 - ETA: 19:07 - loss: 2.2473 - accuracy: 0.390 - ETA: 15:30 - loss: 2.3890 - accuracy: 0.381 - ETA: 13:05 - loss: 2.2196 - accuracy: 0.401 - ETA: 11:21 - loss: 2.0549 - accuracy: 0.442 - ETA: 10:04 - loss: 1.9376 - accuracy: 0.468 - ETA: 9:03 - loss: 1.8418 - accuracy: 0.486 - ETA: 8:15 - loss: 1.7996 - accuracy: 0.50 - ETA: 7:35 - loss: 1.7575 - accuracy: 0.51 - ETA: 7:02 - loss: 1.7009 - accuracy: 0.52 - ETA: 6:34 - loss: 1.6542 - accuracy: 0.53 - ETA: 6:10 - loss: 1.6450 - accuracy: 0.53 - ETA: 5:49 - loss: 1.6434 - accuracy: 0.53 - ETA: 5:31 - loss: 1.6206 - accuracy: 0.54 - ETA: 5:15 - loss: 1.6071 - accuracy: 0.54 - ETA: 5:00 - loss: 1.6581 - accuracy: 0.53 - ETA: 4:48 - loss: 1.6467 - accuracy: 0.53 - ETA: 4:36 - loss: 1.6360 - accuracy:

Train on 13444 samples, validate on 1662 samples
Epoch 1/10
13444/13444 [==============================] - ETA: 1:13:23 - loss: 2.3957 - accuracy: 0.125 - ETA: 37:13 - loss: 2.3871 - accuracy: 0.1094  - ETA: 25:09 - loss: 2.2639 - accuracy: 0.125 - ETA: 19:07 - loss: 2.1640 - accuracy: 0.132 - ETA: 15:30 - loss: 1.9849 - accuracy: 0.225 - ETA: 13:05 - loss: 1.8897 - accuracy: 0.250 - ETA: 11:21 - loss: 1.7491 - accuracy: 0.321 - ETA: 10:04 - loss: 1.6789 - accuracy: 0.367 - ETA: 9:03 - loss: 1.6011 - accuracy: 0.406 - ETA: 8:15 - loss: 1.5023 - accuracy: 0.45 - ETA: 7:35 - loss: 1.4468 - accuracy: 0.47 - ETA: 7:02 - loss: 1.3993 - accuracy: 0.50 - ETA: 6:34 - loss: 1.3278 - accuracy: 0.53 - ETA: 6:10 - loss: 1.2972 - accuracy: 0.55 - ETA: 5:49 - loss: 1.2475 - accuracy: 0.57 - ETA: 5:31 - loss: 1.1928 - accuracy: 0.59 - ETA: 5:15 - loss: 1.1671 - accuracy: 0.60 - ETA: 5:00 - loss: 1.1509 - accuracy: 0.60 - ETA: 4:47 - loss: 1.1350 - accuracy: 0.61 - ETA: 4:36 - loss: 1.1083 - accuracy:

INFO:tensorflow:Oracle triggered exit


In [25]:
tuner.search_space_summary()

In [39]:
tuner.search_space_summary(extended=True)

In [26]:
tuner.results_summary()

In [27]:
models = tuner.get_best_models(num_models=2)

In [ ]:
models

In [28]:
models[0].summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 192)               393408    
_________________________________________________________________
batch_normalization (BatchNo (None, 192)               768       
_________________________________________________________________
re_lu (ReLU)                 (None, 192)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 1158      
Total params: 23,983,046
Trainable params: 23,929,542
Non-trainable params: 53,504
_______________________________________

In [36]:
asd = tuner.get_best_hyperparameters()[0]
type(asd)

kerastuner.engine.hyperparameters.HyperParameters

In [31]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 192)               393408    
_________________________________________________________________
batch_normalization (BatchNo (None, 192)               768       
_________________________________________________________________
re_lu (ReLU)                 (None, 192)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 1158      
Total params: 23,983,046
Trainable params: 23,929,542
Non-trainable params: 53,504
_______________________________________

In [29]:
callback = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto', baseline=None, restore_best_weights=False)]

In [30]:
#trained = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val,y_val))
trained = models[0].fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val,y_val), callbacks=callback)

Train on 13444 samples, validate on 1662 samples
Epoch 1/20
13444/13444 [==============================] - ETA: 1:07:39 - loss: 0.0126 - accuracy: 1.000 - ETA: 34:21 - loss: 0.0076 - accuracy: 1.0000  - ETA: 23:15 - loss: 0.0230 - accuracy: 0.989 - ETA: 17:41 - loss: 0.0178 - accuracy: 0.992 - ETA: 14:21 - loss: 0.0165 - accuracy: 0.993 - ETA: 12:08 - loss: 0.0148 - accuracy: 0.994 - ETA: 10:33 - loss: 0.0160 - accuracy: 0.995 - ETA: 9:21 - loss: 0.0144 - accuracy: 0.996 - ETA: 8:25 - loss: 0.0142 - accuracy: 0.99 - ETA: 7:41 - loss: 0.0138 - accuracy: 0.99 - ETA: 7:04 - loss: 0.0127 - accuracy: 0.99 - ETA: 6:34 - loss: 0.0119 - accuracy: 0.99 - ETA: 6:08 - loss: 0.0112 - accuracy: 0.99 - ETA: 5:46 - loss: 0.0106 - accuracy: 0.99 - ETA: 5:27 - loss: 0.0102 - accuracy: 0.99 - ETA: 5:10 - loss: 0.0099 - accuracy: 0.99 - ETA: 4:55 - loss: 0.0094 - accuracy: 0.99 - ETA: 4:42 - loss: 0.0107 - accuracy: 0.99 - ETA: 4:30 - loss: 0.0102 - accuracy: 0.99 - ETA: 4:19 - loss: 0.0100 - accuracy: 0

KeyboardInterrupt: 

In [ ]:
plot.plot(trained.history['acc'])
plot.plot(trained.history['val_acc'])
plot.title('Model accuracy')
plot.ylabel('Accuracy')
plot.xlabel('Epoch')
plot.legend(['Train', 'Valid'], loc='upper left')
plot.show()

plot.plot(trained.history['loss'])
plot.plot(trained.history['val_loss'])
plot.title('Model loss')
plot.ylabel('Loss')
plot.xlabel('Epoch')
plot.legend(['Train', 'Valid'], loc='upper left')
plot.show()

In [ ]:
# test_images,test_labels = get_images('./test/')
# test_images = np.array(test_images)
# test_labels = np.array(test_labels)
model.evaluate(X_test,y_test, verbose=1)

In [ ]:
keras.layers.Dense(192, activation=None)
keras.layers.BatchNormalization()
keras.layers.ReLU()